#	Some tries for PCA and dataset quality

In [1]:
import sys
import lalsimulation as lalsim
import lal
sys.path.insert(1, '/home/stefano/Desktop/Stefano/scuola/uni/tesi_magistrale/code/routines')
from GW_helper import *
import matplotlib.pyplot as plt
from ML_routines import *

In [2]:
    #loading dataset and splitting
theta_vector, amp_dataset, ph_dataset, frequencies = load_dataset("../datasets/GW_TD_dataset.dat", shuffle=False, N_grid = None) #loading dataset

train_frac = .8
train_theta, test_theta, train_amp, test_amp = make_set_split(theta_vector, amp_dataset, train_frac, 1e-21)
train_theta, test_theta, train_ph, test_ph   = make_set_split(theta_vector, ph_dataset, train_frac, 1.)

OSError: ../datasets/GW_TD_dataset.dat not found.

In [ ]:
s = str(train_theta[0,1:])
print("s = ", s)
print(frequencies[1000], frequencies[1600])
%matplotlib notebook
for i in [100,1600]:#,1000,1500,2000]:
    plt.plot(train_theta[:,0], train_amp[:,i], 'o', ms = 2, label = str(i))
plt.legend()
plt.show()

In [ ]:
plt.figure()
index_ = [1,7,24,51,32]
for index in index_:
    plt.plot(frequencies, train_amp[index,:], label = str(train_theta[index,0]))
plt.legend()
plt.show()

In [ ]:
K_amp = 10
ph_PCA = PCA_model()
E = ph_PCA.fit_model(train_amp, K_amp)
print("PCA eigenvalues: ", E)
PCA_train_amp = ph_PCA.reduce_data(train_amp)
PCA_test_amp = ph_PCA.reduce_data(test_amp)

In [ ]:
plt.figure()
plt.title("PCA components vs q")
for i in range(3):
    plt.plot(train_theta[:,0], PCA_train_amp[:,i], 'o', ms = 2, label = str(i))
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("PCA components vs s1")
for i in range(4):
    #plt.plot(frequencies, ph_PCA.get_V_matrix()[:,i], label= str(i))
    plt.plot(train_theta[:,1], PCA_train_amp[:,i], 'o', ms = 2, label = str(i))
plt.legend()
plt.show()

In [ ]:
#computing mismatches
rec_test_amp = ph_PCA.reconstruct_data(np.multiply(PCA_test_amp,max_amp))
F = compute_mismatch(test_amp, test_ph, rec_test_amp, test_ph)
print("Avg mismatch: ", np.mean(F))

Checking if it's all right with the waves

In [ ]:
#choosing wafeform to compare
indices = 9
print("Waveform chosen: ", train_theta[indices,:])

In [ ]:
print("Generating wave with: ", train_theta[indices,:])
f_high = 1000; f_step = 5e-2; f_low = 20; d = 1.
m2 = 10.
m1 = train_theta[indices,0]*m2
spin1z = train_theta[indices,1]
spin2z = train_theta[indices,2]
LALpars = lal.CreateDict()
approx = lalsim.SimInspiralGetApproximantFromString("IMRPhenomPv2")

hptilde, hctilde = lalsim.SimInspiralChooseFDWaveform( #where is its definition and documentation????
    m1*lalsim.lal.MSUN_SI, #m1
    m2*lalsim.lal.MSUN_SI, #m2
    0, 0, spin1z, #spin vector 1
    0, 0, spin2z, #spin vector 2
    d*1e6*lalsim.lal.PC_SI, #distance to source
    0, #inclination
    0, #something I don't know
    0, #longAscNodes
    0, #eccentricity
    0, #meanPerAno
    f_step, # frequency incremental step
    f_low, # lowest value of frequency
    f_high, # highest value of frequency
    f_low, #some reference value of frequency (??)
    LALpars, #some lal dictionary
    approx #approx method for the model
    )
K = int((f_high-f_low)/f_step)
full_frequencies = np.arange(f_low, f_high, f_step)
h = np.array(hptilde.data.data)+1j*np.array(hctilde.data.data)
temp_amp = (np.abs(h)[int(f_low/f_step):int(f_high/f_step)].real)
temp_ph = (np.unwrap(np.angle(h))[int(f_low/f_step):int(f_high/f_step)].real)
temp_ph = temp_ph - temp_ph[0]
h = h[int(f_low/f_step):int(f_high/f_step)]
(index,) = np.where(temp_amp/temp_amp[0] < 5e-3)
if len(index) >0:
    temp_ph[index] = temp_ph[index[0]-1]

In [ ]:
print((1700-f_low)/f_step, h.shape)
print(temp_amp.shape)
plt.figure()
#plt.plot(full_frequencies, h.real, 'o', ms = 1)
plt.plot(full_frequencies, 1e21*temp_amp, 'o', ms = 1)
plt.plot(frequencies, train_amp[indices,:],'o',ms = 1)
plt.xscale("log")
plt.show()

In [ ]:
plt.figure()

new_ph = np.interp(full_frequencies, frequencies, train_ph[indices,:])
new_amp = np.interp(full_frequencies, frequencies, train_amp[indices,:])

#plt.plot(frequencies, train_amp[indices,:]*np.exp(1j*train_ph[indices,:]), 'o', ms = 2)
plt.plot(full_frequencies, new_amp*np.exp(1j*new_ph), 'o', ms = 1,label = "rec" )
plt.plot(full_frequencies, 1e21*temp_amp*np.exp(1j*temp_ph), 'o', ms = 1, label = "true")
plt.legend()
plt.xscale("log")
plt.show()

In [ ]:
#computing mismatch
print("Mismatch: ", compute_mismatch(new_amp,new_ph, temp_amp, temp_ph)[0])